# $BlurVision: \space AI-Based \space ROI \space Protection  \space for \space  Copyright \space  Compliance$

# $Setup$ ✅
---



In [ ]:
import cv2
import os
import shutil
import numpy as np
import yaml
import torch
# Check for CUDA device and set it
torch.cuda.set_device(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
# checks
from ultralytics import YOLO, checks, hub
checks()

In [3]:
import supervision as sv
from supervision.draw.color import ColorPalette
from supervision import Detections, BoxAnnotator
smoother = sv.DetectionsSmoother()

In [4]:
from supervision import ColorLookup

# $Model$ $Selection$ 🍇
---


In [5]:
from ultralytics import YOLO
model = YOLO("yolov8x-seg.pt").to(device)  # load a pretrained model (recommended for training)

# $Region$ $of $ $Interest$ $Selection$

In [ ]:
SOURCE_VIDEO_PATH='Videos\7293938-hd_1920_1080_30fps.mp4'
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
video_info

## $Utils$

In [52]:
def get_bbox_center(box):
    """Calculate the center point coordinates of a bounding box.
    Args:
        box (numpy.ndarray): A numpy array containing the bounding box coordinates in the form (xmin, ymin, xmax, ymax).

    Returns:
        tuple: A tuple containing the coordinates of the center point in the form (center_x, center_y).
    """
    box=box.astype(int)
    center_x = (box[0] + box[2]) / 2
    center_y = (box[1] + box[3]) / 2
    return np.array([center_x, center_y])

In [53]:
def is_point_inside_bounding_box(point, bounding_box):
    # Unpack the point and bounding box coordinates
    x, y = point
    x_min, y_min, x_max, y_max = bounding_box

    # Check if the point is inside the bounding box
    if x_min <= x <= x_max and y_min <= y <= y_max:
        return True
    else:
        return False



In [112]:
# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, frame, rectangle_defined, rectangle_coords

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            frame_copy = frame.copy()
            cv2.rectangle(frame_copy, (ix, iy), (x, y), (0, 255, 0), 4)
            cv2.imshow('Video', frame_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rectangle_defined = True
        rectangle_coords = (ix, iy, x, y)

## $RoI \space Selection$

In [113]:
# ---------------------------- Region of Interest Selction by User ---------------------------
tracker = sv.ByteTrack(frame_rate=video_info.fps)

# Initialize variables
drawing = False                 # True if the mouse is pressed
ix, iy = -1, -1                 # Initial position of the rectangle
rectangle_defined = False       # To check if a rectangle has been defined
rectangle_coords = None         # To store the coordinates of the defined rectangle


cap2 = cv2.VideoCapture(SOURCE_VIDEO_PATH)  # Replace with 1 for the second camera


if not cap2.isOpened():
    print("Error opening video stream or file")

cv2.namedWindow('Video')
cv2.setMouseCallback('Video', draw_rectangle)


original_width=video_info.width
original_height=video_info.width

scale_x = original_width / 920
scale_y = original_height / 560


# Read until video is completed
while cap2.isOpened():
    # Capture frame-by-frame
    ret, frame = cap2.read()
    if ret:
        
        # Resize the frame
        
        result=model.track(frame,classes=[0])[0]
        detections = sv.Detections.from_ultralytics(result)
        detections = tracker.update_with_detections(detections)
        
        # bounding_box_annotator = sv.BoundingBoxAnnotator(color_lookup=ColorLookup.TRACK)
        # frame = bounding_box_annotator.annotate(
        #     scene=frame.copy(),
        #     detections=detections
        # )
        # Display the frame
        frame = cv2.resize(frame, (1300, 700))
        cv2.imshow('Video', frame)
        if cv2.waitKey(100000) & 0xFF == ord('q'):
            break
    else:
        break
# When everything done, release the video capture object
cap2.release()
# Closes all the frames
cv2.destroyAllWindows()
rectangle_coords=list(rectangle_coords)
rectangle_coords[0] *= scale_x  # Scale x coordinates
rectangle_coords[2] *= scale_x  # Scale x coordinates
rectangle_coords[1] *= scale_y  # Scale y coordinates
rectangle_coords[3] *= scale_y  # Scale y coordinates
rectangle_coords=np.array(rectangle_coords).astype(int)
print(f"Rectangle coordinates: {rectangle_coords}")


Rectangle coordinates: [ 254  178  874 2362]


## $Processing$

In [131]:

cap2 = cv2.VideoCapture(SOURCE_VIDEO_PATH)  # Replace with 1 for the second camera
# Video Saver / Video Writer - results()
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(f'blured.mp4', fourcc, video_info.fps, (video_info.width, video_info.height))

if not cap2.isOpened():
    print("Error opening video stream or file")

cv2.namedWindow('Video')
first_attempt = True

while cap2.isOpened():
    ret, frame = cap2.read()
    if ret:
        
        result = model.track(frame, classes=[0],verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)
        detections = tracker.update_with_detections(detections)

        if first_attempt:
            key = [tracker_id
                   for xyxy, _, _, _, tracker_id, _, in detections 
                   if is_point_inside_bounding_box(get_bbox_center(xyxy), rectangle_coords)]
            print(key)
            key = key[0]
            first_attempt = False
        
        new_classes = np.array([1 if key == ID else 0 for ID in detections.tracker_id])
        detections.class_id = new_classes

        # Filtering detections to keep only the ROI
        selected_classes = [1] 
        detections = detections[np.isin(detections.class_id, selected_classes)]

        # Assuming detection mask exists for the ROI
        if hasattr(detections, 'mask') :
            if len(detections.mask)==0:
                mask=np.zeros((1080,1920))
            else:
                # Remove the extra dimension from the mask
                mask = np.squeeze(detections.mask.astype(np.uint8))  # Now (560, 920)

            # Ensure the mask is single-channel and matches the frame size
            if mask.shape != frame.shape[:2]:
                mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]))

            # Step 1: Dilate the mask to expand it slightly
            kernel = np.ones((15, 15), np.uint8)  # Adjust as needed
            mask = cv2.dilate(mask, kernel, iterations=1)
            mask=mask.astype(float)


            # Step 2: Create a gradient (feather) effect by applying a large Gaussian blur
            feathered_mask = cv2.GaussianBlur(mask, (31,31), 0)

            # Normalize the feathered mask to range between 0 and 1
            # feathered_mask = feathered_mask.astype(float) / 255.0

            # Apply blur to the whole image
            blurred_frame = cv2.GaussianBlur(frame, (7, 7), 100,100)


            # Step 3: Blend the original image and blurred image using the feathered mask
            final_frame = (frame * feathered_mask[..., np.newaxis] + 
                           blurred_frame * (1 - feathered_mask[..., np.newaxis])).astype(np.uint8)
        else:
            final_frame = frame


        out.write(final_frame)
        final_frame = cv2.resize(final_frame, (920, 560))
        cv2.imshow('Video', final_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the video capture object
cap2.release()
out.release()
# Close all the frames
cv2.destroyAllWindows()


[452]
